In [4]:
model_path = r'models\fl_fedavg\effnet-b0\ff_effnet0_1fc_fl.pth'
video_path = r'000.mp4'
#image_path = r'test_original.png'

In [5]:
import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
from efficientnet_pytorch import EfficientNet

from facenet_pytorch import MTCNN
from PIL import Image
import cv2
import numpy as np

device = 'cuda'


IMG_SIZE = 224


mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

class ImageTransform:
    def __init__(self, size, mean, std, train=False):
        self.data_transform = transforms.Compose([
            transforms.Resize((size, size), interpolation=Image.BILINEAR),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])

    def __call__(self, img):
        return self.data_transform(img)



transformer = ImageTransform(IMG_SIZE, mean, std, train=False)

In [6]:
mtcnn = MTCNN(margin=300, select_largest=False, factor=0.5, device=device, post_process=False) # post_process=False if want human readable image
cap = cv2.VideoCapture(video_path)

frame_interval = 32
frame_idx = 0
predictions = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    if frame_idx % frame_interval == 0:
        # Convert frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_img = Image.fromarray(frame_rgb)

        # ====== 3. Detect faces ======
        faces = mtcnn(pil_img)

        if faces is None:
            print(f"[Frame {frame_idx}] No face detected.")
        else:
            # Ensure list of faces
            if isinstance(faces, torch.Tensor):
                faces = [faces]

            for i, face in enumerate(faces):
                face_tensor = transform(face).unsqueeze(0).to(device)

                with torch.no_grad():
                    output = torch.sigmoid(model(face_tensor))
                    prob = output.item()
                    predicted_class = int(prob > 0.5)

                predictions.append({
                    'frame': frame_idx,
                    'face_idx': i,
                    'prob': prob,
                    'predicted_class': predicted_class
                })

                print(f"[Frame {frame_idx} - Face {i+1}] Class: {predicted_class}, Prob: {prob:.4f}")

    frame_idx += 1

cap.release()

c:\Users\Sheryl\AppData\Local\Programs\Python\Python312\Lib\site-packages\facenet_pytorch\models\mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torc

ValueError: Coordinate 'right' is less than 'left'